In [1]:
import requests
from bs4 import BeautifulSoup
import os
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

In [ ]:
### Download and save each file from the website
# 8 downloads can run parallel (improving speed)

index_url = "https://trein.fwrite.org/idx/dedup_OVFiets.html"
response = requests.get(index_url)
soup = BeautifulSoup(response.text, "html.parser")

links = [a["href"] for a in soup.find_all("a", href=True) if a["href"].endswith(".csv.xz")]

os.makedirs("ovfiets_2023", exist_ok=True)

# Filter to only include files from a certain date onward
filtered_links = []
for link in links:
    basename = os.path.basename(link)
    # Expecting something like OVFiets_YYYYMMDD.csv.xz
    try:
        date_str = basename.split("_")[1].split(".")[0].replace("-", "")  # "YYYYMMDD"
        if date_str >= "20240315":  # Change this for the startdate
            filtered_links.append(link)
    except Exception:
        continue

def download_file(link):
    if link.startswith("http"):
        file_url = link
    else:
        file_url = "https://trein.fwrite.org/" + link.lstrip("./").lstrip("../")
    
    filename = os.path.join("ovfiets_2023", os.path.basename(link))
    
    print(f"Downloading {filename} ...")
    r = requests.get(file_url)
    
    if r.status_code == 200:
        with open(filename, "wb") as f:
            f.write(r.content)
        return filename
    else:
        print(f"⚠️ Skipped {file_url} (status {r.status_code})")
        return None

# Download files in parallel
with ThreadPoolExecutor(max_workers=8) as executor:  # Adjust max_workers based on your connection
    downloaded_files = list(executor.map(download_file, filtered_links))

